In [1]:
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from collections import defaultdict
import pyodbc 
import pandas as pd
import numpy as np
import umap
from sklearn import cluster


In [4]:
RC=pd.read_csv('C:/Users\dmckenzie\OneDrive - DLR Group\Desktop\SurveyDataForCluster.csv')

In [48]:
import nltk
def tokenlenfilter(tokenlist,minlen=4):
    tokens = [x for x in tokenlist if len(x)>minlen]
    return (tokens)

def lemmer(text):
    lemma_function = WordNetLemmatizer()

    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV

    return [lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in pos_tag(text)]

def get_ngrams(series: pd.Series,n:int):
    ngrams = series.copy().str.split(' ').explode()
    ngrams1 = series.copy().str.split(' ').explode()
    for i in range(1,n):
        foo=ngrams1.groupby(level=0).shift(-i)
        ngrams=ngrams.str.cat(foo,sep=' ')
    ngrams = ngrams.dropna()
    return pd.DataFrame(ngrams)

def get_multigrams(series,n):
    grams=get_ngrams(series,1)
    if n==1:
        return grams
    else:
        for i in range(1,n+1):
            if i==1:
                continue
            grams=grams.append(get_ngrams(series,i))
    return grams

In [71]:
RC['Name2']=RC.iloc[:,36].str.lower().str.replace('\d+','').str.replace('\+','').str.replace('[^\w\s]',' ').str.replace('_',' ').\
    str.lstrip().str.rstrip('-').str.rstrip()

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [72]:
def tokennamefilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if x not in ['none','and','nothing','office','the','to','the','i','in']]
    return (tokens)

In [73]:
get_ngrams(RC[RC['Name2'].isna()==False]['Name2'].\
               apply(nltk.word_tokenize).\
               apply(tokennamefilter).\
               apply(tokenlenfilter).\
               apply(' '.join).str.lstrip(),1).value_counts().head(60)

Name2        
easier           43
person           35
people           28
socialize        24
better           22
socializing      17
others           16
meetings         16
working          15
collaborate      14
there            13
think            13
learning         11
around           11
being             9
because           9
would             9
seeing            8
coworkers         8
other             8
socialization     8
workers           7
collaboration     7
talking           7
ideas             6
computer          6
meeting           6
while             6
sometimes         6
everyone          6
works             6
things            6
lunch             5
about             5
which             5
getting           5
circulate         5
interaction       5
training          5
collaborating     5
ability           4
hands             4
concentrate       4
online            4
distractions      4
break             4
learn             4
between           4
difficult         4
relati

In [18]:
RC['NameFilter']=RC['Name2'].apply(nltk.word_tokenize).\
    apply(tokenlenfilter).\
    apply(tokennamefilter).\
    apply(lemmer)

TypeError: expected string or bytes-like object

In [24]:
RC['NameFilter']=RC['NameFilter'].apply(' '.join).str.lstrip()

In [25]:
for i in roomkinds:
    print(i.upper())
    print('\n')
    print(RC[RC['Label2']==i]['NameFilter'].value_counts().head(10))
    print('\n')

FAIL


mullion rectangular          49148
locker single                16072
elbow                         8944
                              7263
door ext hmframe              6119
receptacle duplex eef lvl     6043
flange wide                   5980
chair standard zuma inch      5779
elbow round                   4894
chair pantoswing              4646
Name: NameFilter, dtype: int64


CLASSROOM


mullion rectangular                   18907
chair standard zuma inch               6457
chair pantoswing                       6006
desk student                           4769
                                       4471
markerboard tackboard                  3383
dimension lumber                       2848
upper cabinet dbl door                 2820
base cabinet open vertical divider     2565
seat node series stool mid back        2434
Name: NameFilter, dtype: int64


CORRIDOR


locker single               17712
elbow                        4660
transition                   3393
mullion rect

chair pantoswing                        196
                                        189
receptacle duplex eef lvl               184
elbow round                             114
elbow                                    95
recess troffer ceiling flat lens llf     82
classroom chair ruckus task              72
metal shelf                              72
telecomm outlet sdd lvl                  63
csd wall panel                           63
Name: NameFilter, dtype: int64


SHOWER


shower                 205
drain floor            163
transition             101
curtain shower          99
elbow                   96
hook robe               90
locker single           87
                        79
tee                     79
taboret shower trim     70
Name: NameFilter, dtype: int64


CUSTODIAL


elbow                              342
transition                         150
tee                                125
mopsink sink                       100
bend pvc sch dwv                    78
elbow ro

In [26]:
filtercriteria=(RC['Name2'].str.contains('door')==False) & \
    (RC['Name2'].str.contains('enscape')==False) & \
    (RC['Category'].str.contains('Structural')==False) & \
    (RC['Category'].str.contains('Fittings')==False) & \
    (RC['Category']!='Doors') & \
    (RC['Category']!='Walls') & \
    (RC['Category']!='Ceilings') & \
    (RC['Category']!='Roofs') & \
    (RC['Category']!='Stairs') & \
    (RC['Category']!='Columns') & \
    (RC['Category']!='Floors') 

In [27]:
len(RC['Category'].value_counts())

44

In [28]:
forjoin=RC[filtercriteria][RC['NameFilter']!=''].drop_duplicates(subset=['NameFilter'])['NameFilter']
namelines=RC[filtercriteria][RC['NameFilter']!=''].drop_duplicates(subset=['NameFilter'])['Name']

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [29]:
len(RC[filtercriteria][RC['Name']!=''].drop_duplicates(subset=['Name'])['Name'])

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


15412

In [30]:
len(forjoin)

7544

In [31]:
import umap
from sklearn import cluster

In [32]:
# import necessary libraries 
import tensorflow_hub as hub 
  
# Load pre-trained universal sentence encoder model 
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") 
  
# Sentences for which you want to create embeddings, 
# passed as an array in embed() 
embeddings = embed(list(forjoin)) 


In [33]:
reducer = umap.UMAP( n_components=3)
embedding = reducer.fit_transform(embeddings)
bedded2=pd.DataFrame({'X1':embedding[:,0],
                      "X2":embedding[:,1],
                      "X3":embedding[:,2]})

In [56]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=25)
cluster_labels = clusterer.fit_predict(bedded2)

In [57]:
import plotly.express as px 
df = pd.DataFrame(embedding)
fig = px.parallel_coordinates(
    df,
    color=cluster_labels,
    dimensions=[0,1,2],
    color_continuous_scale=px.colors.diverging.Tealrose
)
  
fig.show()

In [58]:
import plotly.express as px 
  
df = px.data.iris() 
  
fig = px.scatter_3d( x = embedding[:,0],  
                    y = embedding[:,1],  
                    z = embedding[:,2],
                    opacity=0.7,
                    hover_name=namelines, 
                    color=cluster_labels.astype('str')) 
fig.update_traces(marker=dict(size=4),
                  selector=dict(mode='markers'))
  
fig.show()

In [37]:
ClusterFrame=pd.DataFrame({'Cluster':cluster_labels.astype('str'),'Item':forjoin})
#print(len(ClusterFrame))
#topitem=ClusterFrame[ClusterFrame['Cluster'].isin(topvalues)].reset_index()['Item']
from gensim.summarization import keywords
ClusterFrame['Cluster']='a'+ClusterFrame['Cluster']
labelframe=ClusterFrame.groupby(['Cluster'])['Item'].\
    apply(lambda x: '. '.join(x)).\
    reset_index()
labelframe['Label']=labelframe['Item'].\
    apply(keywords,args=(2,3)).str.replace('\n',' ')
labelframe=labelframe[['Cluster','Label']]
ClusterFrame=ClusterFrame.set_index('Cluster').\
    join(labelframe.set_index('Cluster'),how='left')[['Item','Label']].\
    reset_index()


In [38]:
labelframe

,Cluster,Label
0,a-1,seat seating seated wall chair
1,a0,dividend horizon pedestal
2,a1,ssd camera wall
3,a10,int fix wdw
4,a100,floor flooring lvl box
...,...,...
110,a95,panel end room
111,a96,wall panel tack
112,a97,single double combo
113,a98,sink countertop counter


In [39]:
#filtercriteria=(RC['Name2'].str.contains('door')==False) & \
#    (RC['Name2'].str.contains('enscape')==False) & \
#    (RC['Category'].str.contains('Structural')==False) & \
#    (RC['Category'].str.contains('Fittings')==False) & \
#    (RC['Category']!='Doors') & \
#    (RC['Category']!='Walls') & \
#    (RC['Category']!='Ceilings') & \
#    (RC['Category']!='Columns') & \
#    (RC['Category']!='Floors') 

In [40]:
RC2=RC[filtercriteria].set_index('NameFilter').join(ClusterFrame.\
            rename(columns={'Item':'NameFilter'}).\
        set_index('NameFilter'),how='inner').\
    reset_index()

In [41]:
len(RC2[RC2['Cluster']!='a-1'].drop_duplicates(subset=['RoomId']))

18343

In [42]:
topitem=RC2[RC2['Cluster']!='a-1'].\
    assign(TopItemCount=1).\
    groupby(['Cluster','Label2','Name'])['TopItemCount'].\
    sum().\
    reset_index().\
    sort_values(['Cluster','Label2','TopItemCount'],ascending=False).\
    groupby(['Cluster','Label2']).\
    head(1)

In [43]:
RC2[RC2['Cluster']!='a-1'].\
    drop_duplicates(subset=['RoomId','Cluster']).\
    assign(dummy=1).\
    groupby(['Label2','Label','Cluster'])['dummy'].\
    sum().\
    reset_index().\
    sort_values(['Label2','dummy'],ascending=False).\
    groupby(['Label2']).\
    head(10).\
    query('Label2!="Fail"')

,Label2,Label,Cluster,dummy
1917,storage,sensor ceiling detector,a13,272
1931,storage,supply diffuser grille grill,a47,259
1898,storage,light switch lld lighting,a99,207
1909,storage,receptacle duplex eef,a5,203
1869,storage,ceiling light llf,a103,187
...,...,...,...,...
124,a.v.,data outlet sdd,a92,34
161,a.v.,table round rectangle,a57,29
133,a.v.,floor flooring lvl box,a100,24
149,a.v.,scd wall speaker,a101,22


In [44]:
tops=RC2[RC2['Cluster']!='a-1'].\
    drop_duplicates(subset=['RoomId','Cluster']).\
    assign(dummy=1).\
    groupby(['Label2','Label','Cluster'])['dummy'].\
    sum().\
    reset_index().\
    sort_values(['Label2','dummy'],ascending=False).\
    set_index(['Label2']).\
    join(RC2.drop_duplicates(subset=['RoomId']).\
         assign(RoomTypeTotal=1).\
         groupby(['Label2'])['RoomTypeTotal'].\
         sum().\
         reset_index().\
         set_index(['Label2'])).\
    reset_index().\
    assign(RoomClusterPorp=lambda x: round((x.dummy / x.RoomTypeTotal),2)*100).\
    set_index(['Cluster']).\
    join(RC2[RC2['Cluster']!='a-1'].\
    drop_duplicates(subset=['RoomId','Cluster']).\
    assign(ClustTotal=1).\
    groupby(['Cluster'])['ClustTotal'].\
    sum().reset_index().\
    sort_values(['ClustTotal'],ascending=False).\
    set_index(['Cluster'])).\
    reset_index().\
    assign(PopTotal=lambda x: round((x.ClustTotal/len(RC2[RC2['Cluster']!='a-1'].drop_duplicates(subset=['RoomId']))),2)*100).\
    assign(Delta=lambda x: x.RoomClusterPorp-x.PopTotal).\
    sort_values(['Label2','Delta'],ascending=False).\
    groupby(['Label2']).\
    head(10).set_index(['Cluster','Label2']).join(topitem[['Cluster','Name','Label2']].set_index(['Cluster','Label2'])).reset_index()

In [45]:
tops2=tops.rename(columns ={'Label2':'Room Type',
                      'Label':'Auto Cluster Label',
                     'dummy':'# Rooms With At Least 1(Within Room Type)',
                     'RoomTypeTotal':'# Room Type',
                     'RoomClusterPorp':'% Rooms With At Least 1(Within Room Type)',
                     'ClustTotal':'# Rooms With At Least 1(Population)',
                     'PopTotal':'% Rooms With At Least 1(Population)',
                     'Name':'Most Common Component With Cluster(Within Room Type)'}).drop(columns=['Cluster'])

In [46]:
tops2=tops2[tops2['Room Type']!='Fail']

In [47]:
tops2.to_csv('Desktop\DollKitsPassOne.csv')

In [48]:
tops[tops['Label2']=='art']

,Cluster,Label2,Label,dummy,RoomTypeTotal,RoomClusterPorp,ClustTotal,PopTotal,Delta,Name
210,a98,art,sink countertop counter,164,263,62.0,4562,25.0,37.0,Sink_Counter_22x19_DLR
211,a113,art,markerboard tltaccy tltaccys casework,151,263,57.0,5786,32.0,25.0,DHT-AP1830E
212,a57,art,table round rectangle,102,263,39.0,3482,19.0,20.0,Table_Mooreco_Hierarchy-Shapes-Standard_DLR
213,a53,art,cart shelf mobile,53,263,20.0,878,5.0,15.0,Shelves_Mobile_Rectangle_DLR
214,a65,art,drawer cabinet base,52,263,20.0,1430,8.0,12.0,TMI D5851 Base Cabinet 5 Drawer
215,a90,art,tech corridor television,57,263,22.0,1871,10.0,12.0,Television_Flat-Screen_DLR
216,a101,art,scd wall speaker,61,263,23.0,2151,12.0,11.0,31300 (Green)
217,a38,art,chair floor beanbag,47,263,18.0,1211,7.0,11.0,VS_PantoSwing Chair_31400
218,a8,art,ottoman eef hex,44,263,17.0,1393,8.0,9.0,Ottoman_Rocker_NorvaNivel_DLR
219,a21,art,student cubby lsi,27,263,10.0,406,2.0,8.0,TMI W9031 Student Cubby Triple_bottomcubby


In [49]:
tops[tops['Label2']=='bathroom']

,Cluster,Label2,Label,dummy,RoomTypeTotal,RoomClusterPorp,ClustTotal,PopTotal,Delta,Name
200,a14,bathroom,dispenser mount wall,1108,1250,89.0,3716,20.0,69.0,DLR_Toilet_Tissue_Dispenser
201,a78,bathroom,flush mount toilet,1046,1250,84.0,2894,16.0,68.0,WC_Flush-Valve_Wall-mounted_DLR
202,a17,bathroom,grab bar barre,990,1250,79.0,2675,15.0,64.0,DLR_GrabBar_WC_Back_GB-1
203,a15,bathroom,vertical horizontal tltaccy,758,1250,61.0,3062,17.0,44.0,DLR_GrabBar_WC_Vertical_GB-3
204,a105,bathroom,floor drain square,597,1250,48.0,1816,10.0,38.0,DLR_Floor_Drain
205,a85,bathroom,ada handicapped handicap clearance,573,1250,46.0,1692,9.0,37.0,DLR_Wheel_Chair_Circle_2D
206,a84,bathroom,ada toilet shower,536,1250,43.0,1558,8.0,35.0,DLR_Lavatory_ADA
207,a81,bathroom,urinal ipf toilet,511,1250,41.0,2504,14.0,27.0,Urinal_Face-hosted_DLR
208,a87,bathroom,toilet standard brace,406,1250,32.0,1430,8.0,24.0,American Standard - Lucerne Lavatory
209,a86,bathroom,toilet accessible brace,300,1250,24.0,1334,7.0,17.0,Toilet_Stall_Accessible_Front_DLR


In [50]:
tops[tops['Label2']=='classroom']

,Cluster,Label2,Label,dummy,RoomTypeTotal,RoomClusterPorp,ClustTotal,PopTotal,Delta,Name
190,a113,classroom,markerboard tltaccy tltaccys casework,1496,2861,52.0,5786,32.0,20.0,DLR_MarkerBoard_TackBoard
191,a98,classroom,sink countertop counter,1127,2861,39.0,4562,25.0,14.0,DLR_Sink_Counter_22x19
192,a90,classroom,tech corridor television,653,2861,23.0,1871,10.0,13.0,Television_Flat-Screen_DLR
193,a57,classroom,table round rectangle,765,2861,27.0,3482,19.0,8.0,Steelcase Coalesse - Akira - Table - Rectangle...
194,a62,classroom,desk office student,401,2861,14.0,1302,7.0,7.0,DLR_Desk_Student_3D
195,a21,classroom,student cubby lsi,267,2861,9.0,406,2.0,7.0,Double_Student_Wardrobe
196,a26,classroom,chair task leg,541,2861,19.0,2234,12.0,7.0,Chair_Task_Mika_DLR
197,a53,classroom,cart shelf mobile,353,2861,12.0,878,5.0,7.0,Shelves_Mobile_Rectangle_DLR
198,a82,classroom,projector screen ceiling,276,2861,10.0,624,3.0,7.0,Projector-Sony-VPL-SW536M
199,a65,classroom,drawer cabinet base,411,2861,14.0,1430,8.0,6.0,PDX_LSI1062_Cabinet_Base_36inch_Sgl_Drawer


In [51]:
tops[tops['Label2']=='corridor']

,Cluster,Label2,Label,dummy,RoomTypeTotal,RoomClusterPorp,ClustTotal,PopTotal,Delta,Name
160,a39,corridor,cabinet recess cab,335,972,34.0,1541,8.0,26.0,Fire_Extinguisher_Cabinet_Semi-Recessed_DLR
161,a94,corridor,exit sign ceiling,318,972,33.0,1457,8.0,25.0,Egress_Path_DLR
162,a9,corridor,locker tier double,227,972,23.0,1067,6.0,17.0,Locker_Single_DLR
163,a1,corridor,ssd camera wall,165,972,17.0,737,4.0,13.0,DLRz_SSD_Wall-Ceiling-Camera
164,a18,corridor,valve inch schematic,150,972,15.0,1061,6.0,9.0,DLR Ball Valve
165,a100,corridor,floor flooring lvl box,201,972,21.0,2720,15.0,6.0,DLR_J-Box-no_model_EEF_LVL
166,a101,corridor,scd wall speaker,179,972,18.0,2151,12.0,6.0,DLR_Intercom_Speaker_Ceiling_SCD_LVL
167,a13,corridor,sensor ceiling detector,361,972,37.0,5607,31.0,6.0,DLR_Smoke_Detector_Clg-Grid_SFA_LVL
168,a10,corridor,int fix wdw,89,972,9.0,817,4.0,5.0,Int_HMframe_Fixed_DLR
169,a41,corridor,mech equipment heater,90,972,9.0,1108,6.0,3.0,-EF-MECH-EQMT-L-300


In [52]:
tops[tops['Label2']=='science']

,Cluster,Label2,Label,dummy,RoomTypeTotal,RoomClusterPorp,ClustTotal,PopTotal,Delta,Name
20,a98,science,sink countertop counter,164,303,54.0,4562,25.0,29.0,Sink_Counter_22x19_DLR
21,a65,science,drawer cabinet base,94,303,31.0,1430,8.0,23.0,TMI D3001 Base Cabinet Tpl Drawer Equal
22,a39,science,cabinet recess cab,82,303,27.0,1541,8.0,19.0,DLR_Fire_Extinguisher_Cabinet_Recessed
23,a25,science,stool seat round,78,303,26.0,1404,8.0,18.0,DLR_Lab_Stool
24,a73,science,shower eyewash corp,57,303,19.0,435,2.0,17.0,EyeWash_Shower_3D_DLR
25,a76,science,science table counter,49,303,16.0,272,1.0,15.0,Science Peninsula
26,a42,science,unit gas meter,53,303,17.0,594,3.0,14.0,Gas-Shutoff_DLR
27,a57,science,table round rectangle,99,303,33.0,3482,19.0,14.0,DLR_Table_Rectangular_3D
28,a52,science,hood microwave kiln,36,303,12.0,377,2.0,10.0,Fume_Hood_3523_DLR
29,a113,science,markerboard tltaccy tltaccys casework,125,303,41.0,5786,32.0,9.0,DLR_MarkerBoard_TackBoard


In [53]:
tops[tops['Label2']=='lab']

,Cluster,Label2,Label,dummy,RoomTypeTotal,RoomClusterPorp,ClustTotal,PopTotal,Delta,Name
110,a42,lab,unit gas meter,80,292,27.0,594,3.0,24.0,Gas
111,a98,lab,sink countertop counter,130,292,45.0,4562,25.0,20.0,Sink_Counter_22x19_DLR
112,a113,lab,markerboard tltaccy tltaccys casework,148,292,51.0,5786,32.0,19.0,DLR_MarkerBoard_TackBoard
113,a25,lab,stool seat round,69,292,24.0,1404,8.0,16.0,DLR_Lab_Stool
114,a65,lab,drawer cabinet base,64,292,22.0,1430,8.0,14.0,TMI D3001 Base Cabinet Tpl Drawer Equal
115,a39,lab,cabinet recess cab,60,292,21.0,1541,8.0,13.0,DLR_Fire_Extinguisher_Cabinet_Recessed
116,a46,lab,rectangular electrical electric balance,92,292,32.0,3708,20.0,12.0,Rectangular Mullion
117,a62,lab,desk office student,49,292,17.0,1302,7.0,10.0,DLR_Desk_Student_2D
118,a76,lab,science table counter,31,292,11.0,272,1.0,10.0,041011 Science Rectangle Table
119,a57,lab,table round rectangle,86,292,29.0,3482,19.0,10.0,DLR_Table_Rectangular_3D


In [54]:
tops[tops['Label2']=='princi']

,Cluster,Label2,Label,dummy,RoomTypeTotal,RoomClusterPorp,ClustTotal,PopTotal,Delta,Name
30,a26,princi,chair task leg,79,124,64.0,2234,12.0,52.0,Chair-Task
31,a57,princi,table round rectangle,54,124,44.0,3482,19.0,25.0,Steelcase - C Scape - Shared Table - Tapered
32,a62,princi,desk office student,33,124,27.0,1302,7.0,20.0,Desk1
33,a47,princi,supply diffuser grille grill,50,124,40.0,4910,27.0,13.0,+BMS_MCH_Generic_Diffuser
34,a61,princi,workstation ped assistant,13,124,10.0,209,1.0,9.0,Workstation_Ped_DLR
35,a108,princi,lighting light recess lead,31,124,25.0,3032,17.0,8.0,+BMS_LT_Generic_2x2_Recessed_Troffer
36,a5,princi,receptacle duplex eef,40,124,32.0,4487,24.0,8.0,Receptacle_Duplex_EEF_LVL_DLR
37,a65,princi,drawer cabinet base,20,124,16.0,1430,8.0,8.0,DLR_Cabinet-File 4 Drawer
38,a13,princi,sensor ceiling detector,46,124,37.0,5607,31.0,6.0,ME-Ceiling Fire Alarm
39,a36,princi,series seat chair,10,124,8.0,315,2.0,6.0,Steelcase - Seating - Leap 462 Series - Work C...


In [55]:
tops[tops['Label2']=='gym']

,Cluster,Label2,Label,dummy,RoomTypeTotal,RoomClusterPorp,ClustTotal,PopTotal,Delta,Name
130,a2,gym,basketball court basket,51,154,33.0,88,0.0,33.0,DLR_VolleyBall_Court
131,a20,gym,gym wall window,45,154,29.0,131,1.0,28.0,GYM Panels
132,a39,gym,cabinet recess cab,51,154,33.0,1541,8.0,25.0,Fire_Extinguisher_Cabinet_Semi-Recessed_DLR
133,a7,gym,flex bleacher table,39,154,25.0,96,1.0,24.0,DLR_Bleachers - Flex
134,a93,gym,wall extension void,40,154,26.0,698,4.0,22.0,DLR_Wall_Pad
135,a94,gym,exit sign ceiling,43,154,28.0,1457,8.0,20.0,Exit Sign
136,a100,gym,floor flooring lvl box,47,154,31.0,2720,15.0,16.0,DLR_J-Box-no_model_EEF_LVL
137,a1,gym,ssd camera wall,28,154,18.0,737,4.0,14.0,DLRz_SSD_Wall-Ceiling-Camera
138,a5,gym,receptacle duplex eef,55,154,36.0,4487,24.0,12.0,DLR_Receptacle_Duplex_EEF_LVL
139,a79,gym,acoustic acoustical panel wall,20,154,13.0,266,1.0,12.0,Acoustic_Wall_Panel_DLR


In [ ]:
tops[tops['Label2']=='bathroom'][['Label2','Name']].append(topany[(topany['Label2']=='bathroom') & 
                                                                 (topany['Name'].\
                                                                  isin(list(tops[tops['Label2']=='bathroom']['Name']))==False)]  )

In [ ]:
tops[tops['Label2']=='bathroom'][['Label2','Name']].append(topany[(topany['Label2']=='bathroom') & 
                                                                 (topany['Name'].\
                                                                  isin(list(tops[tops['Label2']=='bathroom']['Name']))==False)]  )

In [ ]:
topany[(topany['Label2']=='bathroom') & 
                                                                 topany['Name'].\
                                                                  isin(list(tops[tops['Label2']=='bathroom']['Name']))==False]

In [ ]:
RC2['Category'].value_counts()

In [ ]:
RC2[RC2['Cluster']!='a-1'].\
    assign(TopItemCount=1).\
    groupby(['Label','Category'])['TopItemCount'].\
    sum().\
    reset_index().\
    sort_values(['Category','TopItemCount'],ascending=False).\
    groupby(['Category']).\
    head(1)

In [ ]:
topany=RC2[RC2['Label2']!='Fail'].assign(TopItemCount=1).\
    groupby(['Label2','Name'])['TopItemCount'].\
    sum().\
    reset_index().\
    sort_values(['Label2','TopItemCount'],ascending=False).\
    groupby(['Label2']).\
    head(10)[['Label2','Name']]